In [294]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


import sklearn

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC


import xgboost as xgb


from sklearn import tree
from tensorflow import keras

In [295]:
class Preprocessing():
    
    def pre_processing(self,dataset):
        #print(dataset.isnull().sum())
        dataset['Gender'] = dataset['Gender'].map({'Male':1,'Female':0})
        dataset['Married'] = dataset['Married'].map({'Yes':1,'No':0})
        dataset['Education'] = dataset['Education'].map({'Graduate':1,'Not Graduate':0})
        dataset['Self_Employed'] = dataset['Self_Employed'].map({'Yes':1,'No':0})
        dataset['Property_Area'] = dataset['Property_Area'].map({'Urban':1,'Rural':0,'Semiurban':2})
        dataset['Dependents'] = dataset['Dependents'].map({'0':1,'1':0,'2':2,'3+':3})
        
        return dataset
    
    def adjust_null(self,dataset):
        '''
        dataset['Gender'].fillna('Male', inplace=True)
        dataset['Married'].fillna('Yes', inplace=True)
        dataset['Dependents'].fillna(0, inplace=True)
        dataset['Self_Employed'].fillna('No', inplace=True)
        dataset['Credit_History'].fillna(2, inplace=True)
        '''
        dataset['LoanAmount'].fillna(dataset['LoanAmount'].median(), inplace = True)
        dataset['Loan_Amount_Term'].fillna(dataset['Loan_Amount_Term'].mode()[0], inplace=True)
        
        dataset['Gender'].fillna(dataset['Gender'].mode()[0], inplace=True)
        dataset['Married'].fillna(dataset['Married'].mode()[0], inplace=True)
        dataset['Dependents'].fillna(dataset['Dependents'].mode()[0], inplace=True)
        dataset['Self_Employed'].fillna(dataset['Self_Employed'].mode()[0], inplace=True)
        dataset['Credit_History'].fillna(dataset['Credit_History'].mode()[0], inplace=True)
        
        #print(dataset.isnull().sum())
        return dataset
        
        
        
    def data_analysis(self):
        sns.barplot(x = 'Gender', y = 'Loan_Status', data=self.train)
        sns.barplot(x = 'ApplicantIncome', y = 'Loan_Status',data=self.train)
        sns.barplot(x = 'Self_Employed', y = 'Loan_Status', data=self.train)
        #plt.scatter(self.x_train, self.y_train)
        #plt.show()
        
    def get_preprocess_data(self,data):
        dataset = self.pre_processing(data)
        dataset = self.adjust_null(dataset)
        #dataset['Loan_Status'] = self.label_encoder.fit_transform(dataset['Loan_Status'])
        dataset['Loan_Status'] = dataset['Loan_Status'].map({'Y':1,'N':0})
        #print(dataset.head())
        #print(len(dataset))
        dataset = dataset.dropna(axis=0)
        dataset = dataset.drop('Loan_ID',axis=1)
        #print(dataset.isnull().sum())
        #print(len(dataset))
        self.train,self.test = train_test_split(dataset,test_size = 0.1,random_state = 0,shuffle=False)
        
        self.y_train = self.train['Loan_Status']
        self.x_train = self.train.drop('Loan_Status',axis=1)
        
        
        self.y_test = self.test['Loan_Status']
        self.x_test = self.test.drop('Loan_Status',axis=1)
        return dataset
        #print(dataset.describe())

In [299]:
class GenModel():
    def __init__(self):
        self.model = None
        self.x_train = preprocess_obj.x_train
        self.y_train = preprocess_obj.y_train
        self.x_test = preprocess_obj.x_test
        self.y_test = preprocess_obj.y_test
        
    def train(self):
        self.model1 = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
        
        self.model2 = GradientBoostingClassifier(learning_rate=0.01,random_state=1)

        self.model3 = LogisticRegression(class_weight='balanced')
        
        #self.model4 = xgb.XGBClassifier(random_state=1,learning_rate=0.01)
        self.model4 = xgb.XGBClassifier(random_state=1, n_estimators=81, max_depth=1)

        self.model5 = AdaBoostClassifier(random_state=1,n_estimators=5)

        self.model6 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        
        self.model7 = MLPClassifier(hidden_layer_sizes=(3), max_iter=100)
        
        self.model8 = RandomForestClassifier(random_state=1, max_depth=10, n_estimators=10)

        self.model1.fit(self.x_train,self.y_train)
        print("Bagging: ")
        print(self.model1.score(self.x_test,self.y_test))
        print("\n")
        
        self.model2.fit(self.x_train,self.y_train)
        print("Gradient Boosting: ")
        print(self.model2.score(self.x_test,self.y_test))
        self.model2.fit(self.x_test,self.y_test)
        print(self.model2.score(self.x_test,self.y_test))
        print("\n")
        
        self.model3.fit(self.x_train,self.y_train)
        print("Logistic Regression: ")
        print(self.model3.score(self.x_test,self.y_test))
        print("\n")
        
        print("XGB: ")
        self.model4.fit(self.x_train,self.y_train)
        print(self.model4.score(self.x_test,self.y_test))
        self.model4.fit(self.x_test,self.y_test)
        print(self.model4.score(self.x_test,self.y_test))
        print("\n")
        
        print("AdaBoost: ")
        self.model5.fit(self.x_train,self.y_train)
        print(self.model5.score(self.x_test,self.y_test))
        self.model5.fit(self.x_test,self.y_test)
        print(self.model5.score(self.x_test,self.y_test))
        print("\n")
        
        print("KNN: ")
        self.model6.fit(self.x_train,self.y_train)
        print(self.model6.score(self.x_test,self.y_test))
        print("\n")
        
        print("MLP: ")
        self.model7.fit(self.x_train,self.y_train)
        print(self.model7.score(self.x_test,self.y_test))
        print("\n")
        
        print("Random Forest: ")
        self.model8.fit(self.x_train,self.y_train)
        print(self.model8.score(self.x_test,self.y_test))
        
    def nn_model1(self):
        n_cols = self.x_train.shape[1]
        self.model = keras.models.Sequential()
    
        self.model.add(keras.layers.Dense(256, kernel_initializer="truncated_normal", activation = "relu", input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
    
        self.model.add(keras.layers.Dense(128, kernel_initializer="truncated_normal", activation = "relu", input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
    
        self.model.add(keras.layers.Dense(64, kernel_initializer="truncated_normal", activation = "relu", input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
    
        self.model.add(keras.layers.Dense(1, kernel_initializer="truncated_normal", activation = "sigmoid" ))
    
        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ["accuracy"])
        self.model.fit(self.x_train,self.y_train, batch_size = 10, epochs=50, validation_data = (self.x_test,self.y_test))
    
    def nn_model2(self):
        n_cols = self.x_train.shape[1]
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.Dense(128, activation='relu', input_shape = (n_cols,)))
        self.model.add(keras.layers.Dense(64, activation='relu'))
        self.model.add(keras.layers.Dense(32, activation='relu'))
        self.model.add(keras.layers.Dense(1, activation='sigmoid'))

        self.model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
        self.model.fit(self.x_train,self.y_train, batch_size = 5,epochs=50, validation_data = (self.x_test,self.y_test))
        

In [300]:
train_data = pd.read_csv(os.path.join('Data','train.csv'))
preprocess_obj = Preprocessing()
train_data = preprocess_obj.get_preprocess_data(train_data)

In [301]:
model_obj = GenModel()
model_obj.train()

Bagging: 
0.8225806451612904


Gradient Boosting: 
0.8387096774193549
0.9193548387096774




/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Logistic Regression: 
0.7741935483870968


XGB: 
0.8387096774193549
0.9516129032258065


AdaBoost: 
0.8548387096774194
0.8870967741935484


KNN: 
0.5806451612903226


MLP: 
0.7096774193548387


Random Forest: 
0.7741935483870968


In [287]:
model_obj.nn_model2()

Train on 552 samples, validate on 62 samples
Epoch 1/50
552/552 [==============================] - 0s 828us/sample - loss: 39.9995 - accuracy: 0.5688 - val_loss: 9.8522 - val_accuracy: 0.5323
Epoch 2/50
552/552 [==============================] - 0s 260us/sample - loss: 19.3756 - accuracy: 0.5471 - val_loss: 19.0642 - val_accuracy: 0.4355
Epoch 3/50
552/552 [==============================] - 0s 265us/sample - loss: 16.6818 - accuracy: 0.5598 - val_loss: 35.9035 - val_accuracy: 0.3710
Epoch 4/50
552/552 [==============================] - 0s 260us/sample - loss: 14.4795 - accuracy: 0.5815 - val_loss: 11.2875 - val_accuracy: 0.5645
Epoch 5/50
552/552 [==============================] - 0s 254us/sample - loss: 7.5805 - accuracy: 0.5942 - val_loss: 6.0655 - val_accuracy: 0.7097
Epoch 6/50
552/552 [==============================] - 0s 336us/sample - loss: 6.2644 - accuracy: 0.6214 - val_loss: 12.3674 - val_accuracy: 0.4355
Epoch 7/50
552/552 [==============================] - 0s 273us/sample -

In [302]:
test_data = pd.read_csv(os.path.join('Data','test.csv'),index_col='Loan_ID') 
print(test_data.head())
test_data.isnull().sum()

#test_data['Dependents'].unique() 
#pd.unique(test_data[['Dependents', 'Loan_Amount_Term']].values)

         Gender Married Dependents     Education Self_Employed  \
Loan_ID                                                          
LP001015   Male     Yes          0      Graduate            No   
LP001022   Male     Yes          1      Graduate            No   
LP001031   Male     Yes          2      Graduate            No   
LP001035   Male     Yes          2      Graduate            No   
LP001051   Male      No          0  Not Graduate            No   

          ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
Loan_ID                                                                      
LP001015             5720                  0       110.0             360.0   
LP001022             3076               1500       126.0             360.0   
LP001031             5000               1800       208.0             360.0   
LP001035             2340               2546       100.0             360.0   
LP001051             3276                  0        78.0             

Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [292]:
class Prediction():
    def __init__(self):
        self.model = model_obj.model4
        
    def adjust_null(self,train,test):
        test['Gender'].fillna(train['Gender'].mode()[0], inplace=True)
        test['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True)
        test['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True)
        test['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)
        test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
        test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)
        return test
    def prediction(self,test_data):
        test_data = preprocess_obj.pre_processing(test_data)
        test_data = self.adjust_null(train_data,test_data)
        print(test_data.isnull().sum())

        
        y_predict=self.model.predict(test_data)
        y_predict=pd.DataFrame(y_predict,columns =['Loan_Status'],index=test_data.index)
        y_predict['Loan_Status']= y_predict['Loan_Status'].map({1: 'Y', 0: 'N'})
        y_predict.to_csv("Submission.csv")
        print(y_predict.head())

In [293]:
pred_obj = Prediction()
pred_obj.prediction(test_data)

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64
         Loan_Status
Loan_ID             
LP001015           Y
LP001022           Y
LP001031           Y
LP001035           N
LP001051           Y
